In [21]:
!pip install -q transformers datasets accelerate bitsandbytes peft evaluate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.8 MB/s eta 0:00:00


In [22]:
from datasets import load_dataset
import datasets
import pandas as pd
print(dir(datasets))

['Array2D', 'Array3D', 'Array4D', 'Array5D', 'ArrowBasedBuilder', 'Audio', 'BuilderConfig', 'ClassLabel', 'Column', 'Dataset', 'DatasetBuilder', 'DatasetDict', 'DatasetInfo', 'DownloadConfig', 'DownloadManager', 'DownloadMode', 'Features', 'GeneratorBasedBuilder', 'Image', 'IterableColumn', 'IterableDataset', 'IterableDatasetDict', 'LargeList', 'List', 'NamedSplit', 'NamedSplitAll', 'Pdf', 'ReadInstruction', 'Sequence', 'Split', 'SplitBase', 'SplitDict', 'SplitGenerator', 'SplitInfo', 'StreamingDownloadManager', 'SubSplitInfo', 'Translation', 'TranslationVariableLanguages', 'Value', 'VerificationMode', 'Version', 'Video', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'are_progress_bars_disabled', 'arrow_dataset', 'arrow_reader', 'arrow_writer', 'builder', 'combine', 'concatenate_datasets', 'config', 'data_files', 'dataset_dict', 'disable_caching', 'disable_progress_bar', 'disable_progress_bars', 'doc

In [23]:
data=load_dataset('SetFit/bbc-news')
print(data['train'][:5])

README.md:   0%|          | 0.00/880 [00:00<?, ?B/s]

train.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1225 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'text': ['wales want rugby league training wales could follow england s lead by training with a rugby league club.  england have already had a three-day session with leeds rhinos  and wales are thought to be interested in a similar clinic with rivals st helens. saints coach ian millward has given his approval  but if it does happen it is unlikely to be this season. saints have a week s training in portugal next week  while wales will play england in the opening six nations match on 5 february.  we have had an approach from wales   confirmed a saints spokesman.  it s in the very early stages but it is something we are giving serious consideration to.  st helens  who are proud of their welsh connections  are obvious partners for the welsh rugby union  despite a spat in 2001 over the collapse of kieron cunningham s proposed £500 000 move to union side swansea. a similar cross-code deal that took iestyn harris from leeds to cardiff in 2001 did go through  before the talented stand-off ret

In [24]:
train_df = pd.DataFrame(data["train"])
test_df = pd.DataFrame(data["test"])

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

print(train_df.head())
print(test_df.head())

Train shape: (1225, 3)
Test shape: (1000, 3)
                                                text  label     label_text
0  wales want rugby league training wales could f...      2          sport
1  china aviation seeks rescue deal scandal-hit j...      1       business
2  rock band u2 break ticket record u2 have smash...      3  entertainment
3  markets signal brazilian recovery the brazilia...      1       business
4  tough rules for ringtone sellers firms that fl...      0           tech
                                                text  label     label_text
0  carry on star patsy rowlands dies actress pats...      3  entertainment
1  sydney to host north v south game sydney will ...      2          sport
2  uk coal plunges into deeper loss shares in uk ...      1       business
3  blair joins school sailing trip the prime mini...      4       politics
4  bath faced with tindall ultimatum mike tindall...      2          sport


In [25]:
label2id = {label: i for i, label in enumerate(train_df["label_text"].unique())}
id2label = {v: k for k, v in label2id.items()}

train_df["label"] = train_df["label_text"].map(label2id)
test_df["label"]  = test_df["label_text"].map(label2id)

In [26]:
train_df.head()

,text,label,label_text
0,wales want rugby league training wales could f...,0,sport
1,china aviation seeks rescue deal scandal-hit j...,1,business
2,rock band u2 break ticket record u2 have smash...,2,entertainment
3,markets signal brazilian recovery the brazilia...,1,business
4,tough rules for ringtone sellers firms that fl...,3,tech


In [27]:
test_df.head()

,text,label,label_text
0,carry on star patsy rowlands dies actress pats...,2,entertainment
1,sydney to host north v south game sydney will ...,0,sport
2,uk coal plunges into deeper loss shares in uk ...,1,business
3,blair joins school sailing trip the prime mini...,4,politics
4,bath faced with tindall ultimatum mike tindall...,0,sport


In [28]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(
    train_df["text"].tolist(),
    truncation=True,
    padding=True,
    max_length=256
)

test_encodings = tokenizer(
    test_df["text"].tolist(),
    truncation=True,
    padding=True,
    max_length=256
)

In [29]:
import torch

class BBCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

train_dataset = BBCDataset(train_encodings, train_df["label"].tolist())
test_dataset  = BBCDataset(test_encodings,  test_df["label"].tolist())

In [30]:
from transformers import BertForSequenceClassification, TrainingArguments, Trainer

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
# Freeze all BERT encoder layers
for param in model.bert.parameters():
    param.requires_grad = False

# Only the classification head (model.classifier) will be trained

In [32]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bbc_bert_linear_probe",
    num_train_epochs=15,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    report_to=[],  # disables W&B and other loggers
)

In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # your PyTorch dataset
    eval_dataset=test_dataset      # your PyTorch dataset
)

In [34]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.625800,1.593910
2,1.588700,1.573798
3,1.571100,1.558607
4,1.561700,1.543958
5,1.546600,1.531771
6,1.536400,1.519995
7,1.526700,1.511443
8,1.511900,1.502852
9,1.509500,1.495103
10,1.506300,1.490134


TrainOutput(global_step=2310, training_loss=1.5293390315332454, metrics={'train_runtime': 599.3128, 'train_samples_per_second': 30.66, 'train_steps_per_second': 3.854, 'total_flos': 2417397933888000.0, 'train_loss': 1.5293390315332454, 'epoch': 15.0})

In [35]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [36]:
from transformers import BertForSequenceClassification, Trainer

# checkpoint_path = "./bbc_bert_linear_probe/checkpoint-1078"
# fine_tuned_model = BertForSequenceClassification.from_pretrained(checkpoint_path)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=test_dataset)
print(metrics)

{'eval_loss': 1.477121353149414, 'eval_model_preparation_time': 0.0199, 'eval_accuracy': 0.395, 'eval_f1': 0.2953519078489126, 'eval_precision': 0.5964570400425059, 'eval_recall': 0.395, 'eval_runtime': 15.2743, 'eval_samples_per_second': 65.47, 'eval_steps_per_second': 8.184}


In [37]:
# Load the last fine-tuned checkpoint
model = BertForSequenceClassification.from_pretrained("/content/bbc_bert_linear_probe/checkpoint-2310")


# Training arguments
training_args = TrainingArguments(
    output_dir="./bbc_bert_linear_probe",
    num_train_epochs=15,             # you can increase epochs for more training
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,              # used internally by Trainer
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    report_to=[]
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()

# Evaluate on test set
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Updated test metrics (pure linear probe):", metrics)

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.612800,0.178815,0.966000,0.965928,0.966722,0.966000
2,0.073000,0.147485,0.962000,0.962498,0.965474,0.962000
3,0.023800,0.109507,0.973000,0.972928,0.972983,0.973000
4,0.007700,0.125616,0.977000,0.977079,0.977707,0.977000
5,0.005500,0.120592,0.976000,0.975995,0.976128,0.976000
6,0.002400,0.122890,0.978000,0.978054,0.978290,0.978000
7,0.001800,0.134940,0.973000,0.973020,0.973205,0.973000
8,0.001500,0.139326,0.974000,0.973979,0.974100,0.974000
9,0.001300,0.141914,0.973000,0.972981,0.973078,0.973000
10,0.001200,0.144100,0.973000,0.972981,0.973078,0.973000


Updated test metrics (pure linear probe): {'eval_loss': 0.1095074787735939, 'eval_accuracy': 0.973, 'eval_f1': 0.9729280219088312, 'eval_precision': 0.9729825657672075, 'eval_recall': 0.973, 'eval_runtime': 12.4466, 'eval_samples_per_second': 80.343, 'eval_steps_per_second': 5.062, 'epoch': 15.0}


In [38]:
train_df['label_text'].unique()

array(['sport', 'business', 'entertainment', 'tech', 'politics'],
      dtype=object)

In [40]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load your fine-tuned model (best checkpoint)
model = BertForSequenceClassification.from_pretrained(
    "/content/bbc_bert_linear_probe/checkpoint-2310"
)
model.eval()  # Set to evaluation mode

# Label mapping (BBC dataset usually has 5 classes)
label_names = list(train_df['label_text'].unique())

def predict(text):
    # Tokenize
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256
    )

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted class
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()

    # Return label name
    return label_names[predicted_class_id]

In [41]:
title=input("Enter your news title here: ")
predict(title)

Enter your news title here: China  has the largest market of semiconductor


'business'